In [1]:
from google.colab import  drive
drive.mount('/content/gdrive')
# mục đích: truy cập vào google drive để lấy dữ liệu hoặc lưu dữ liệu

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!apt-get install zip
# mục đích: cài đặt công cụ nén và giải nén tệp tin có tên là zip 

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zip is already the newest version (3.0-12build2).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [ ]:
!cd 'gdrive/MyDrive' && unzip 'dataEmotion' -d 'dataEmotion'
# mục đích: giải nén dữ liệu huấn luyện vào một thư mục làm việc trong colab 

Streaming output truncated to the last 5000 lines.
  inflating: dataEmotion/dataEmotion/vuive/21461v.png  
  inflating: dataEmotion/dataEmotion/vuive/21462v.png  
  inflating: dataEmotion/dataEmotion/vuive/21463v.png  
  inflating: dataEmotion/dataEmotion/vuive/21464v.png  
  inflating: dataEmotion/dataEmotion/vuive/21465v.png  
  inflating: dataEmotion/dataEmotion/vuive/21466v.png  
  inflating: dataEmotion/dataEmotion/vuive/21467v.png  
  inflating: dataEmotion/dataEmotion/vuive/21468v.png  
  inflating: dataEmotion/dataEmotion/vuive/21469v.png  
 extracting: dataEmotion/dataEmotion/vuive/2146v.png  
 extracting: dataEmotion/dataEmotion/vuive/2147.png  
  inflating: dataEmotion/dataEmotion/vuive/21470v.png  
  inflating: dataEmotion/dataEmotion/vuive/21471v.png  
  inflating: dataEmotion/dataEmotion/vuive/21472v.png  
  inflating: dataEmotion/dataEmotion/vuive/21473v.png  
  inflating: dataEmotion/dataEmotion/vuive/21474v.png  
  inflating: dataEmotion/dataEmotion/vuive/21475v.png  


In [ ]:
from keras.utils import to_categorical # mã hoá One-Hot
from keras.preprocessing.image import load_img # tải ảnh 
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D # xây dựng kiến trúc CNN
from sklearn.model_selection import train_test_split 
import os
import pandas as pd
import numpy as np

In [ ]:
TRAIN_DIR = "/content/gdrive/MyDrive/dataEmotion/dataEmotion"
# định nghĩa đường dẫn gốc chính xác đến thư mục chứa dữ liệu huấn luyện 
# sau khi đã gắn kết google drive và giải nén tệp 

In [ ]:
# tạo danh sách các đường dẫn tệp và nhãn dãn tương ứng cho toàn bộ dữ liệu huấn luyện 
def createdataframe(dir): 
    image_paths = [] # khởi tạo danh sách rỗng lưu trữ đường dẫn ảnh
    labels = [] # và nhãn cảm xúc tương ứng 
    for label in os.listdir(dir): # duyệt qua từng thư mục con trong thư mục gốc 
        for imagename in os.listdir(os.path.join(dir, label)): # duyệt qua từng tên tệp ảnh 
            image_paths.append(os.path.join(dir, label, imagename)) # xây dựng đường dẫn đầy đủ đến tệp ảnh và thêm vào danh sách
            labels.append(label) # gán nhãn cảm xúc cho tệp tương ứng 
        print(label, "completed") # in ra màn hình khi mỗi quá trình thu thập đường dẫn cho một loại cảm xúc hoàn tất 
    return image_paths, labels # trả về danh sách chứa toàn bộ thông tin cần thiết 

In [ ]:
#tổ chức dữ liệu đầu tiên 
train = pd.DataFrame()
train["image"], train["label"] = createdataframe(TRAIN_DIR)
#Gán hai danh sách trả về vào hai cột mới trong DataFrame train:
# Cột "image": Chứa đường dẫn đầy đủ đến từng tệp ảnh.
# Cột "label": Chứa nhãn cảm xúc dạng chuỗi (ví dụ: 'buon', 'vuive').

batngo completed
binhthuong completed
buon completed
tucgian completed
vuive completed


In [ ]:

from tqdm.notebook import tqdm 
#một thư viện Python nổi tiếng dùng để hiển thị thanh tiến trình

In [ ]:
def extrac_feature(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale = True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48,48,1)
    return features
#Hàm này nhận danh sách đường dẫn ảnh từ DataFrame train
# và chuyển chúng thành định dạng mảng số (NumPy array) 
# sẵn sàng cho mô hình CNN.

In [ ]:
train_features = extrac_feature(train['image'])
#chuyển đổi toàn bộ đường dẫn ảnh thành mảng NumPy,
#  chuẩn bị dữ liệu đầu vào X cho mô hình.

  0%|          | 0/74192 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/keras/src/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [ ]:




X = train_features/255.0
#biến dữ liệu ảnh thô thành dữ liệu định dạng hoàn hảo cho mô hình Model6.h5 

In [ ]:
from sklearn.preprocessing import LabelEncoder
#nhập khẩu công cụ LabelEncoder từ thư viện scikit-learn.
#LabelEncoder được sử dụng để chuyển đổi nhãn dạng văn bản (chuỗi) 
# thành định dạng số nguyên (integer encoding).

In [ ]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [ ]:
Y = le.transform(train['label'])


In [ ]:
Y = to_categorical(Y, num_classes = 5)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=30)

In [ ]:
from keras import Sequential

In [ ]:
model = Sequential()

#Convolution Layers
model.add(Conv2D(32, kernel_size=(3,3), activation = 'relu', input_shape = (48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64, kernel_size=(3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=(3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(256, kernel_size=(3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))



model.add(Flatten())
model.add(Dense(1000,activation = "relu"))

model.add(Dropout(0.3))

model.add(Dense(512,activation = "relu"))
model.add(Dropout(0.2))




model.add(Dense(5, activation = "softmax"))




In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = 'accuracy'
)
# biên dịch mô hình 

In [ ]:
model.fit(x = X_train, y = Y_train, batch_size = 128, epochs = 100, validation_data = (X_test,Y_test))
#Huấn luyện Mô hình (Model Training)

Epoch 1/100
464/464 [==============================] - 22s 16ms/step - loss: 0.7899 - accuracy: 0.6709 - val_loss: 0.4288 - val_accuracy: 0.8284
Epoch 2/100
464/464 [==============================] - 8s 17ms/step - loss: 0.4043 - accuracy: 0.8405 - val_loss: 0.3753 - val_accuracy: 0.8509
Epoch 3/100
464/464 [==============================] - 8s 17ms/step - loss: 0.3485 - accuracy: 0.8628 - val_loss: 0.3587 - val_accuracy: 0.8590
Epoch 4/100
464/464 [==============================] - 7s 15ms/step - loss: 0.3161 - accuracy: 0.8770 - val_loss: 0.3519 - val_accuracy: 0.8641
Epoch 5/100
464/464 [==============================] - 6s 14ms/step - loss: 0.2911 - accuracy: 0.8879 - val_loss: 0.3355 - val_accuracy: 0.8719
Epoch 6/100
464/464 [==============================] - 5s 12ms/step - loss: 0.2642 - accuracy: 0.8977 - val_loss: 0.3319 - val_accuracy: 0.8730
Epoch 7/100
464/464 [==============================] - 6s 14ms/step - loss: 0.2451 - accuracy: 0.9059 - val_loss: 0.3428 - val_accuracy

In [ ]:
model.save("Model7.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
